# Tick 转 Bar

## 转换

Tick 转 Bar 可以很简单，利用 pandas 的 resample 就可以了。前提是 DataFrame 的 index 是 DateTimeIndex 格式。

如：Tick 转 Bar 可以很简单，利用 pandas 的 resample 就可以了。前提是 DataFrame 的 index 是 DateTimeIndex 格式。

如：

In [1]:
from pathlib import Path

import pandas as pd

from src.utility import PACKAGE_PATH


# 定义数据文件
data_file: str = 'SHFE.al2111_Tick.csv'

# 转为Path格式。
data_path: Path = PACKAGE_PATH.joinpath(data_file)

# 加载tick数据为DataFrame。
# 【parse_dates=['datetime']】表示在加载时将 datetime 字段解读为 datetime64 格式。
# 【index_col=['datetime']】表示在加载时将 datetime 字段作为 DateTimeIndex。
df: pd.DataFrame = pd.read_csv(data_path, parse_dates=['datetime'], index_col=['datetime'])

# 显示 df。
print(df)

                                  datetime_nano  last_price  highest   lowest  \
datetime                                                                        
2021-06-30 18:51:42.300000  1625050302300000000         NaN      NaN      NaN   
2021-06-30 20:59:00.500000  1625057940500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:00.500000  1625058000500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.000000  1625058001000000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.500000  1625058001500000000     18780.0  18785.0  18780.0   
...                                         ...         ...      ...      ...   
2021-09-25 00:59:58.000000  1632502798000000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:58.500000  1632502798500000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.000000  1632502799000000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.500000  1632502799500000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.500001  

如果没有在加载时选择 datetimeindex，那么可以稍后转化：

In [2]:
df: pd.DataFrame = pd.read_csv(data_path, parse_dates=['datetime'])
print(df)

                          datetime        datetime_nano  last_price  highest  \
0       2021-06-30 18:51:42.300000  1625050302300000000         NaN      NaN   
1       2021-06-30 20:59:00.500000  1625057940500000000     18780.0  18780.0   
2       2021-06-30 21:00:00.500000  1625058000500000000     18780.0  18780.0   
3       2021-06-30 21:00:01.000000  1625058001000000000     18780.0  18780.0   
4       2021-06-30 21:00:01.500000  1625058001500000000     18780.0  18785.0   
...                            ...                  ...         ...      ...   
1468857 2021-09-25 00:59:58.000000  1632502798000000000     22945.0  23150.0   
1468858 2021-09-25 00:59:58.500000  1632502798500000000     22945.0  23150.0   
1468859 2021-09-25 00:59:59.000000  1632502799000000000     22945.0  23150.0   
1468860 2021-09-25 00:59:59.500000  1632502799500000000     22945.0  23150.0   
1468861 2021-09-25 00:59:59.500001  1632502799500001000     22945.0  23150.0   

          lowest  volume        amount 

In [3]:
df.set_index('datetime', inplace=True)
print(df)

                                  datetime_nano  last_price  highest   lowest  \
datetime                                                                        
2021-06-30 18:51:42.300000  1625050302300000000         NaN      NaN      NaN   
2021-06-30 20:59:00.500000  1625057940500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:00.500000  1625058000500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.000000  1625058001000000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.500000  1625058001500000000     18780.0  18785.0  18780.0   
...                                         ...         ...      ...      ...   
2021-09-25 00:59:58.000000  1632502798000000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:58.500000  1632502798500000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.000000  1632502799000000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.500000  1632502799500000000     22945.0  23150.0  22855.0   
2021-09-25 00:59:59.500001  

转换成 Bar：

In [4]:
# 转成 Bar。
df_1min = df['last_price'].resample('1MIN').ohlc()
print(df_1min)

                        open     high      low    close
datetime                                               
2021-06-30 18:51:00      NaN      NaN      NaN      NaN
2021-06-30 18:52:00      NaN      NaN      NaN      NaN
2021-06-30 18:53:00      NaN      NaN      NaN      NaN
2021-06-30 18:54:00      NaN      NaN      NaN      NaN
2021-06-30 18:55:00      NaN      NaN      NaN      NaN
...                      ...      ...      ...      ...
2021-09-25 00:55:00  22955.0  22960.0  22945.0  22950.0
2021-09-25 00:56:00  22950.0  22960.0  22945.0  22950.0
2021-09-25 00:57:00  22950.0  22955.0  22950.0  22950.0
2021-09-25 00:58:00  22955.0  22960.0  22945.0  22955.0
2021-09-25 00:59:00  22950.0  22960.0  22945.0  22945.0

[124209 rows x 4 columns]


Bar 数据生成了。

可以看出，简单的 resample 之后，DataFrame 有许多不在交易时间中的数据。

这是因为 resample 就是从第一个 tick 时间开始，按照采样参数（本例为一分钟），开始填充数据。

必须加以过滤。

## 过滤

因为：

- 交易所会在非交易时间发送数据
- pandas resample 会生成非交易时间数据

所以需要按照交易时间进行过滤。

判断交易时间：

    开盘时间 <= t <= 收盘时间

国内证券交易时间都分为两至四个“节”。少的比如股票和中金所期货，两个节；没有夜盘的商品期货三个节；其它四个节。

循环，按各节的开盘收盘时间判断即可。

但是，有色、贵金属、原油的夜盘交易时间跨越北京时间24:00，不利于程序判断。

### 时区

因此转为采用虚拟时区，即北京时间（GMT+8）的20:00为虚拟时区（GMT+12）的00:00，从而在一个结算日内，交易时间连续。

时区支持采用 pytz 库，该库是 pandas 的支持库之一。

In [5]:
import datetime as dt
import pytz

# 定义时区。
tz_beijing: pytz.BaseTzInfo = pytz.timezone('Asia/Shanghai')
tz_trading: pytz.BaseTzInfo = pytz.timezone('Etc/GMT-12')       # 为什么是GMT-12？想不明白。

# 时区的 delta。
tz_delta: dt.timedelta = dt.timedelta(hours=4)

# 将 navie 类型（无时区）的 DateTimeIndex 转化为 aware 类型（时区感知）。
df.index = df.index.tz_localize('Asia/Shanghai')
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 18:51:42.300000+08:00  1625050302300000000         NaN      NaN   
2021-06-30 20:59:00.500000+08:00  1625057940500000000     18780.0  18780.0   
2021-06-30 21:00:00.500000+08:00  1625058000500000000     18780.0  18780.0   
2021-06-30 21:00:01+08:00         1625058001000000000     18780.0  18780.0   
2021-06-30 21:00:01.500000+08:00  1625058001500000000     18780.0  18785.0   
...                                               ...         ...      ...   
2021-09-25 00:59:58+08:00         1632502798000000000     22945.0  23150.0   
2021-09-25 00:59:58.500000+08:00  1632502798500000000     22945.0  23150.0   
2021-09-25 00:59:59+08:00         1632502799000000000     22945.0  23150.0   
2021-09-25 00:59:59.500000+08:00  1632502799500000000     22945.0  23150.0   
2021-09-25 00:59:59.500001+08:00  1632502799500001000     22945.

DateTimeIndex 后面有了“+08:00”的字样，表示加上了时区，且确实为北京时间。

转换时区：

In [6]:
df.index = df.index.tz_convert('Etc/GMT-12')
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 22:51:42.300000+12:00  1625050302300000000         NaN      NaN   
2021-07-01 00:59:00.500000+12:00  1625057940500000000     18780.0  18780.0   
2021-07-01 01:00:00.500000+12:00  1625058000500000000     18780.0  18780.0   
2021-07-01 01:00:01+12:00         1625058001000000000     18780.0  18780.0   
2021-07-01 01:00:01.500000+12:00  1625058001500000000     18780.0  18785.0   
...                                               ...         ...      ...   
2021-09-25 04:59:58+12:00         1632502798000000000     22945.0  23150.0   
2021-09-25 04:59:58.500000+12:00  1632502798500000000     22945.0  23150.0   
2021-09-25 04:59:59+12:00         1632502799000000000     22945.0  23150.0   
2021-09-25 04:59:59.500000+12:00  1632502799500000000     22945.0  23150.0   
2021-09-25 04:59:59.500001+12:00  1632502799500001000     22945.

DateTimeIndex 后面有了“+12:00”的字样，时区转化成功。

### 过滤

使用

    开盘时间 <= t <= 收盘时间

的办法，结合 pandas 的 bool index，以 SHFE.al 为例：

In [7]:
def drop_non_trading_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop data in non-trading time.
    """
    if df.index.inferred_type == 'datetime64':
        return df[
            (
                ((df.index.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]
    else:
        return df[
            (
                ((df.datetime.dt.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]

df = drop_non_trading_data(df)
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-07-01 00:59:00.500000+12:00  1625057940500000000     18780.0  18780.0   
2021-07-01 01:00:00.500000+12:00  1625058000500000000     18780.0  18780.0   
2021-07-01 01:00:01+12:00         1625058001000000000     18780.0  18780.0   
2021-07-01 01:00:01.500000+12:00  1625058001500000000     18780.0  18785.0   
2021-07-01 01:00:02+12:00         1625058002000000000     18780.0  18795.0   
...                                               ...         ...      ...   
2021-09-25 04:59:58+12:00         1632502798000000000     22945.0  23150.0   
2021-09-25 04:59:58.500000+12:00  1632502798500000000     22945.0  23150.0   
2021-09-25 04:59:59+12:00         1632502799000000000     22945.0  23150.0   
2021-09-25 04:59:59.500000+12:00  1632502799500000000     22945.0  23150.0   
2021-09-25 04:59:59.500001+12:00  1632502799500001000     22945.

效果不错。

只是，drop_non_trading_data() 中的时间都被写死了，应该是可以根据品种改变的。

### 交易时间

各品种交易时间由查询交易所网站公告信息获得。

目前，各市场各品种都采用“交易节”的形式。比如：

- 股票，2个交易节，<09:30 ~ 11:30, 13:00 ~ 15:00>
- 金融期货，2个交易节
    - 股指期货，<09:30 ~ 11:30, 13:00 ~ 15:00>
    - 国债期货，<09:30 ~ 11:30, 13:00 ~ 15:15>
- 商品期货
    - 无夜盘，3个交易节，<09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
    - 有夜盘，4个交易节：
        - 贵金属、原油：<21:00 ~ 02:30, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        - 有色：<21:00 ~ 01:00, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        - 其它：<21:00 ~ 23:00, 09:00 ~ 10:15, 10:30 ~ 11:30, 13:30 ~ 15:00>
        
以 SHFE.al 为例，整合为一个数据结构：

In [8]:
product_trading_time = {
    'SHFE.al': {
        'count': 4,
        'optional': 1,
        'trading_section': [
            dt.time(hour=21, minute=0, tzinfo=tz_beijing),
            dt.time(hour=1, minute=0, tzinfo=tz_beijing),
            dt.time(hour=9, minute=0, tzinfo=tz_beijing),
            dt.time(hour=10, minute=15, tzinfo=tz_beijing),
            dt.time(hour=10, minute=30, tzinfo=tz_beijing),
            dt.time(hour=11, minute=30, tzinfo=tz_beijing),
            dt.time(hour=13, minute=30, tzinfo=tz_beijing),
            dt.time(hour=15, minute=0, tzinfo=tz_beijing),
        ],
    },
}

print(1, product_trading_time)

print(2, product_trading_time['SHFE.al']['trading_section'][3].isoformat())

1 {'SHFE.al': {'count': 4, 'optional': 1, 'trading_section': [datetime.time(21, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(1, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(9, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 15, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(11, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(13, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>)]}}
2 10:15:00


定义为一个 Python 类：

In [9]:
from typing import List

class ProductTradingTime:
    """
    品种交易时间。
    """

    def __init__(self,
                 exchange: str,
                 product: str,
                 count: int,
                 optional: int,
                 section_list: List[dt.time]
                 ) -> None:

        # 交易所
        self.exchange: str = exchange.upper()

        # 品种
        self.product: str = product

        # 交易节的数量
        if count <= 0:
            raise ValueError(f'Parameter <count> should be positive integer. Got {count} instead.')
        else:
            self.count: int = count

        # 可选的交易节序号（有夜盘为1，无夜盘为0——不可选）。
        if optional < 0 or optional > 1:
            raise ValueError(f'Parameter <optional> should be 0 or 1. Got {optional} instead.')
        else:
            self.optional: int = optional

        # 交易节时间
        if len(section_list) / count != 2:
            raise ValueError(f'Parameter <trading_section> should has twice number items of <count>. {product}')
        else:
            self.section_list: List[dt.time] = section_list

简单测试一下，还是用 SHFE.al 做例子：

In [10]:
test: ProductTradingTime = ProductTradingTime(
    exchange='SHFE',
    product='al',
    count=4,
    optional=1,
    section_list=[
        dt.time(hour=21, minute=0, tzinfo=tz_beijing),
        dt.time(hour=1, minute=0, tzinfo=tz_beijing),
        dt.time(hour=9, minute=0, tzinfo=tz_beijing),
        dt.time(hour=10, minute=15, tzinfo=tz_beijing),
        dt.time(hour=10, minute=30, tzinfo=tz_beijing),
        dt.time(hour=11, minute=30, tzinfo=tz_beijing),
        dt.time(hour=13, minute=30, tzinfo=tz_beijing),
        dt.time(hour=15, minute=0, tzinfo=tz_beijing),
    ]
)

print(f'test.exchange: {test.exchange}')
print(f'test.product: {test.product}')
print(f'test.count: {test.count}')
print(f'test.optional: {test.optional}')
print(f'test.section_list: {test.section_list}')

test.exchange: SHFE
test.product: al
test.count: 4
test.optional: 1
test.section_list: [datetime.time(21, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(1, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(9, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 15, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(11, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(13, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>)]


改进一下，用 property 改写为只读属性：

In [11]:
class ProductTradingTime:
    """
    品种交易时间。
    """
    _exchange: str  # 交易所
    _product: str   # 品种
    _count: int     # 交易节的数量
    _optional: int  # 可选的交易节序号（有夜盘为1，无夜盘为0——不可选）。
    _section_list: List[dt.time]    # 交易节时间

    def __init__(self,
                 exchange: str,
                 product: str,
                 count: int,
                 optional: int,
                 section_list: List[dt.time]
                 ) -> None:

        self._exchange = exchange.upper()

        self._product = product

        if count <= 0:
            raise ValueError(f'Parameter <count> should be positive integer. Got {count} instead.')
        else:
            self._count = count

        if optional < 0 or optional > 1:
            raise ValueError(f'Parameter <optional> should be 0 or 1. Got {optional} instead.')
        else:
            self._optional = optional

        if len(section_list) / count != 2:
            raise ValueError(f'Parameter <trading_section> should has twice number items of <count>. {product}')
        else:
            self._section_list = section_list
    
    @property
    def exchange(self) -> str:
        return self._exchange
    
    @property
    def product(self) -> str:
        return self._product
    
    @property
    def symbol(self) -> str:
        return self._symbol
    
    @property
    def count(self) -> int:
        return self._count
    
    @property
    def optional(self) -> int:
        return self._optional
    
    @property
    def section_list(self) -> int:
        return self._section_list

还是得测试一下：

In [15]:
test: ProductTradingTime = ProductTradingTime(
    exchange='SHFE',
    product='al',
    count=4,
    optional=1,
    section_list=[
        dt.time(hour=21, minute=0, tzinfo=tz_beijing),
        dt.time(hour=1, minute=0, tzinfo=tz_beijing),
        dt.time(hour=9, minute=0, tzinfo=tz_beijing),
        dt.time(hour=10, minute=15, tzinfo=tz_beijing),
        dt.time(hour=10, minute=30, tzinfo=tz_beijing),
        dt.time(hour=11, minute=30, tzinfo=tz_beijing),
        dt.time(hour=13, minute=30, tzinfo=tz_beijing),
        dt.time(hour=15, minute=0, tzinfo=tz_beijing),
    ]
)

print(f'test.exchange: {test.exchange}')
print('-' * 20)
print(f'test.product: {test.product}')
print('-' * 20)
print(f'test.count: {test.count}')
print('-' * 20)
print(f'test.optional: {test.optional}')
print('-' * 20)
print(f'test.section_list: {test.section_list}')
print('-' * 20)
print(f'test: {test}')

test.exchange: SHFE
--------------------
test.product: al
--------------------
test.count: 4
--------------------
test.optional: 1
--------------------
test.section_list: [datetime.time(21, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(1, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(9, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 15, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(11, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(13, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>)]
--------------------
test: <__main__.ProductTradingTime object at 0x0000019FC465C7F0>


最后一行的代码不够好，简直不是给人看的。改进一下：

In [26]:
class ProductTradingTime:
    """
    品种交易时间。
    """
    _exchange: str  # 交易所
    _product: str   # 品种
    _count: int     # 交易节的数量
    _optional: int  # 可选的交易节序号（有夜盘为1，无夜盘为0——不可选）。
    _section_list: List[dt.time]    # 交易节时间

    def __init__(self,
                 exchange: str,
                 product: str,
                 count: int,
                 optional: int,
                 section_list: List[dt.time]
                 ) -> None:

        self._exchange = exchange.upper()

        self._product = product

        if count <= 0:
            raise ValueError(f'Parameter <count> should be positive integer. Got {count} instead.')
        else:
            self._count = count

        if optional < 0 or optional > 1:
            raise ValueError(f'Parameter <optional> should be 0 or 1. Got {optional} instead.')
        else:
            self._optional = optional

        if len(section_list) / count != 2:
            raise ValueError(f'Parameter <trading_section> should has twice number items of <count>. {product}')
        else:
            self._section_list = section_list
    
    @property
    def exchange(self) -> str:
        return self._exchange
    
    @property
    def product(self) -> str:
        return self._product
    
    @property
    def symbol(self) -> str:
        return self._symbol
    
    @property
    def count(self) -> int:
        return self._count
    
    @property
    def optional(self) -> int:
        return self._optional
    
    @property
    def section_list(self) -> int:
        return self._section_list
    
    def __str__(self) -> str:
        return f'<ProductTradingTime(' \
               f'Exchange={self.exchange}, ' \
               f'Product={self.product}, ' \
               f'Count={self.count}, ' \
               f'Optional={self.optional}, ' \
               f'SectionsList={self.section_list}' \
               f')>'
    
    @property
    def show_section_list(self) -> List[str]:
        result: List[str] = []
        for item in self.section_list:
            result.append(item.isoformat())
        return result


test: ProductTradingTime = ProductTradingTime(
    exchange='SHFE',
    product='al',
    count=4,
    optional=1,
    section_list=[
        dt.time(hour=21, minute=0, tzinfo=tz_beijing),
        dt.time(hour=1, minute=0, tzinfo=tz_beijing),
        dt.time(hour=9, minute=0, tzinfo=tz_beijing),
        dt.time(hour=10, minute=15, tzinfo=tz_beijing),
        dt.time(hour=10, minute=30, tzinfo=tz_beijing),
        dt.time(hour=11, minute=30, tzinfo=tz_beijing),
        dt.time(hour=13, minute=30, tzinfo=tz_beijing),
        dt.time(hour=15, minute=0, tzinfo=tz_beijing),
    ]
)


print(f'test: \n{test}')
print('-' * 20)
print(test.show_section_list)

test: 
<ProductTradingTime(Exchange=SHFE, Product=al, Count=4, Optional=1, SectionsList=[datetime.time(21, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(1, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(9, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 15, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(10, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(11, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(13, 30, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>), datetime.time(15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>)])>
--------------------
['21:00:00', '01:00:00', '09:00:00', '10:15:00', '10:30:00', '11:30:00', '13:30:00', '15:00:00']


In [13]:
def drop_non_trading_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop data in non-trading time.
    """
    if df.index.inferred_type == 'datetime64':
        return df[
            (
                ((df.index.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]
    else:
        return df[
            (
                ((df.datetime.dt.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]

df = drop_non_trading_data(df)
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-07-01 00:59:00.500000+12:00  1625057940500000000     18780.0  18780.0   
2021-07-01 01:00:00.500000+12:00  1625058000500000000     18780.0  18780.0   
2021-07-01 01:00:01+12:00         1625058001000000000     18780.0  18780.0   
2021-07-01 01:00:01.500000+12:00  1625058001500000000     18780.0  18785.0   
2021-07-01 01:00:02+12:00         1625058002000000000     18780.0  18795.0   
...                                               ...         ...      ...   
2021-09-25 04:59:58+12:00         1632502798000000000     22945.0  23150.0   
2021-09-25 04:59:58.500000+12:00  1632502798500000000     22945.0  23150.0   
2021-09-25 04:59:59+12:00         1632502799000000000     22945.0  23150.0   
2021-09-25 04:59:59.500000+12:00  1632502799500000000     22945.0  23150.0   
2021-09-25 04:59:59.500001+12:00  1632502799500001000     22945.